## Task 2: **Data Improvement & Further Analysis**

Merging all industry sectors, and analyze based on the merging result

In [1]:
import os
import re
import sys
import json
import nltk
import pandas as pd
import numpy as np
from scipy.stats import norm
from collections import defaultdict
from nltk.stem import WordNetLemmatizer 
import matplotlib.pyplot as plt  

nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def read_data(root_dir, data_folder, csv_file):
    print("\nReading data from " + csv_file)
    file_dir = os.path.join(root_dir, data_folder, csv_file)
    return_df = pd.read_csv(file_dir)
    return return_df

In [0]:
def merge_dataframe(dst_dataframe, new_part):
    no_nan_new_part = new_part.copy().fillna('')
    if dst_dataframe is None:
        dst_dataframe = no_nan_new_part.copy()
        return dst_dataframe
    else:
        dst_dataframe = pd.concat([dst_dataframe, no_nan_new_part])
        return dst_dataframe

In [0]:
def rewrite_industry(all_df, correspond_dict, criterion_dict, name_dict, industry_dict):
    rewritten_df = all_df.copy().drop(labels = ['IndustryName', 'IndustrySegmentName', 'IndustrySectorName'], axis = 1)
    inv_name_dict = {}
    for x in name_dict.items():
        inv_name_dict[x[1]] = x[0]

    mapping = {}
    for company in industry_dict:
        temp = list(industry_dict[company].values())
        infos = temp[0].split('-')
        if len(infos[0].strip()) > 0:
            if (len(infos[1]) == 0) and (len(infos[2]) == 0):
                if infos[0].strip() in correspond_dict:
                    new_class = correspond_dict[infos[0].strip()]
                    mapping[company] = inv_name_dict[new_class]
            elif (len(infos[1]) > 0) and (len(infos[2]) == 0):
                if infos[1].strip() in correspond_dict:
                    new_class = correspond_dict[infos[1].strip()]
                    mapping[company] = inv_name_dict[new_class]
            elif (len(infos[1]) > 0) and (len(infos[2]) > 0):
                if infos[2].strip() in correspond_dict:
                    new_class = correspond_dict[infos[2].strip()]
                    mapping[company] = inv_name_dict[new_class]
            

    sector = []
    sector_name = []
    industry_group = []
    industry_group_name = []
    industry = []
    industry_name = []
    row_idx = 0
    while row_idx < all_df.shape[0]:
        company_kfid = str(all_df.iloc[row_idx, 1])
        if company_kfid in mapping:
            serial_num = str(mapping[company_kfid])
            if len(serial_num) == 2:
                sector.append(serial_num)
                sector_name.append(name_dict[serial_num])
                industry_group.append('')
                industry_group_name.append('')
                industry.append('')
                industry_name.append('')
            elif len(serial_num) == 4:
                sector.append(serial_num[:2])
                sector_name.append(name_dict[serial_num[:2]])
                industry_group.append(serial_num)
                industry_group_name.append(name_dict[serial_num])
                industry.append('')
                industry_name.append('')
            elif len(serial_num) == 6:
                sector.append(serial_num[:2])
                sector_name.append(name_dict[serial_num[:2]])
                industry_group.append(serial_num[:4])
                industry_group_name.append(name_dict[serial_num[:4]])
                industry.append(serial_num)
                industry_name.append(name_dict[serial_num])
            elif len(serial_num) == 8:
                sector.append(serial_num[:2])
                sector_name.append(name_dict[serial_num[:2]])
                industry_group.append(serial_num[:4])
                industry_group_name.append(name_dict[serial_num[:4]])
                industry.append(serial_num[:6])
                industry_name.append(name_dict[serial_num[:6]])
        else:
            sector.append('')
            sector_name.append('')
            industry_group.append('')
            industry_group_name.append('')
            industry.append('')
            industry_name.append('')
        row_idx += 1

    rewritten_df['SectorNumber'] = sector
    rewritten_df['SectorName'] = sector_name
    rewritten_df['IndustryGroupNumber'] = industry_group
    rewritten_df['IndustryGroupName'] = industry_group_name
    rewritten_df['IndustryNumber'] = industry
    rewritten_df['IndustryName'] = industry_name

    return rewritten_df


In [0]:
'''
Basic environ params
'''
# root_dir = "\\".join(os.path.dirname(__file__).split('\\')[:-1])
root_dir = '/content/drive/My Drive/Penn Inequality Project'
data_folder = "reduced_paynet_data"
task_folder = "task2"
data_dir = os.path.join(root_dir, data_folder)

'''
Main adjustable params
'''
# Section of codes, which are listed in every cell
data_merged = True
industry_merged = True
industry_reclassified = True
data_rewritten = True
paying_analysis = False

In [0]:
'''
Merge data into one csv
'''

all_df = None

if not data_merged:
    print('Merging all csv files together')
    for csv_file in os.listdir(data_dir):
        df = read_data(root_dir, data_folder, csv_file)
        all_df = merge_dataframe(all_df, df)
    all_df.to_csv(os.path.join(root_dir, 'all_data.csv'))
    print('All csv files are merged\n')
else:
    print('Reading merged csv files')
    all_df = pd.read_csv(os.path.join(root_dir, 'all_data.csv'))
    print('Including ' + str(all_df.shape[0]) + ' data pieces')

Reading merged csv files


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Including 28397806 data pieces


In [0]:
'''
Merge all industry names based on company and year
'''
industry_dict = None

if not industry_merged:
    print('Merging all industry together')
    industry_dict = defaultdict(dict)

    # key: KF_ID, value: dict of year -- industry_info
    all_companies = pd.unique(all_df['KF_ID'])
    all_years = pd.unique(all_df['CalendarYear'])
    for company in sorted(all_companies):
        company_df = all_df[all_df['KF_ID'] == company]
        for year in sorted(all_years):
            year_company_df = company_df[company_df['CalendarYear'] == year]
            if year_company_df.shape[0] > 0:
                info = year_company_df[['IndustryName', 'IndustrySegmentName', 'IndustrySectorName']].drop_duplicates()
                for row_idx in range(info.shape[0]):
                    # remove special character, multiple spaces, lowercase the string, and remove parenthesis
                    row = info.iloc[row_idx, :]
                    industry_name = re.sub(r'\(.*\)', '', str(row['IndustryName']))
                    industry_name = re.sub(r'\W', ' ', industry_name)
                    industry_name = re.sub(r' +', ' ', industry_name).lower()
                    if len(industry_name) > 0:
                        result = []
                        for word in re.split(' ', industry_name):
                            result.append(lemmatizer.lemmatize(word))
                        industry_name = (' '.join(result)).strip()

                    industry_segment_name = re.sub(r'\(.*\)', '', str(row['IndustrySegmentName']))
                    industry_segment_name = re.sub(r'\W', ' ', industry_segment_name)
                    industry_segment_name = re.sub(r' +', ' ', industry_segment_name).lower()
                    if len(industry_segment_name) > 0:
                        result = []
                        for word in re.split(' ', industry_segment_name):
                            result.append(lemmatizer.lemmatize(word))
                        industry_segment_name = (' '.join(result)).strip()

                    industry_sector_name = re.sub(r'\(.*\)', '', str(row['IndustrySectorName']))
                    industry_sector_name = re.sub(r'\W', ' ', industry_sector_name)
                    industry_sector_name = re.sub(r' +', ' ', industry_sector_name).lower()
                    if len(industry_sector_name) > 0:
                        result = []
                        for word in re.split(' ', industry_sector_name):
                            result.append(lemmatizer.lemmatize(word))
                        industry_sector_name = (' '.join(result)).strip()

                    industry_dict[str(company)][str(year)] = industry_name + '-' + industry_segment_name + '-' + industry_sector_name
    with open(os.path.join(root_dir, task_folder, 'industry_original_dict.json'), 'w') as output_file:
        json.dump(industry_dict, output_file)
else:
    print('Reading merged company files')
    with open(os.path.join(root_dir, task_folder, 'industry_original_dict.json'), 'r') as input_file:
        industry_dict = json.load(input_file)

Reading merged company files


In [0]:
'''
Reclassify industry
'''

name_dict = None
criterion_dict = None
correspond_dict = None

if not industry_reclassified:
    print("Reading new classification criterion")
    criterion = pd.read_excel(os.path.join(root_dir, task_folder, 'GICS_map 2018.xlsx')).drop(index = [0,1,2, 3]).fillna(value = -1)
    criterion_dict = {}
    name_dict = {}
    for row_idx in range(criterion.shape[0]):
        row = criterion.iloc[row_idx, :]
        sector = str(row[0])
        industry_group = str(row[2])
        industry = str(row[4])
        sub_industry = str(row[6]) 

        # remove special character, multiple spaces, and lowercase the string 
        if int(sector) > 0:
            criterion_dict[sector] = {}
            sector_name = re.sub(r'&', ' and ', row[1])
            sector_name = re.sub(r'\W', ' ', sector_name)
            sector_name = re.sub(r' +', ' ', sector_name).lower()
            temp = []
            for word in re.split(' ', sector_name):
                temp.append(lemmatizer.lemmatize(word))
            sector_name = ' '.join(temp)
            name_dict[sector] = re.sub('health care', 'healthcare', sector_name).strip()
        if int(industry_group) > 0:
            criterion_dict[industry_group[:2]][industry_group] = {}
            industry_group_name = re.sub(r'&', ' and ', row[3])
            industry_group_name = re.sub(r'\W', ' ', industry_group_name)
            industry_group_name = re.sub(r' +', ' ', industry_group_name).lower()
            temp = []
            for word in re.split(' ', industry_group_name):
                temp.append(lemmatizer.lemmatize(word))
            industry_group_name = ' '.join(temp)
            name_dict[industry_group] = re.sub('health care', 'healthcare', industry_group_name).strip()
        if int(industry) > 0:
            criterion_dict[industry[:2]][industry[:4]][industry] = {}
            industry_name = re.sub(r'&', ' and ', row[5])
            industry_name = re.sub(r'\W', ' ', industry_name)
            industry_name = re.sub(r' +', ' ', industry_name).lower()
            temp = []
            for word in re.split(' ', industry_name):
                temp.append(lemmatizer.lemmatize(word))
            industry_name = ' '.join(temp)
            name_dict[industry] = re.sub('health care', 'healthcare', industry_name).strip()
        if int(sub_industry) > 0:
            criterion_dict[sub_industry[:2]][sub_industry[:4]][sub_industry[:6]] = sub_industry
            sub_industry_name = re.sub(r'&', ' and ', row[7])
            sub_industry_name = re.sub(r'\W', ' ', sub_industry_name)
            sub_industry_name = re.sub(r' +', ' ', sub_industry_name).lower()
            temp = []
            for word in re.split(' ', sub_industry_name):
                temp.append(lemmatizer.lemmatize(word))
            sub_industry_name = ' '.join(temp)
            name_dict[sub_industry] = re.sub('health care', 'healthcare', sub_industry_name).strip()
    print("\nSaving criterion and name into json files")
    with open(os.path.join(root_dir, task_folder, 'name_dict.json'), 'w') as output_file:  
        json.dump(name_dict, output_file)
    with open(os.path.join(root_dir, task_folder, 'criterion_dict.json'), 'w') as output_file:   
        json.dump(criterion_dict, output_file)



    print("\nReading given samples of classification")
    samples = pd.read_csv(os.path.join(root_dir, task_folder, 'xpf_financials_masked.csv')).iloc[:, :2].drop_duplicates()
    sample_dict = {}
    for x in samples.to_numpy():
        info = re.sub(r'\W', ' ', x[1])
        info = re.sub(r' +', ' ', info).lower()
        temp = []
        for word in re.split(' ', info):
            temp.append(lemmatizer.lemmatize(word))
        sample_dict[str(x[0])] = (' '.join(temp)).strip()
    
    print("\nFinding replacements in orginal classifcation")
    source_dict = defaultdict(list)
    for company in industry_dict:
        if company in list(sample_dict.keys()):
            for year in industry_dict[company]:
                infos = re.split('-', industry_dict[company][year])
                if len(infos[0].strip()) > 0:
                    if (len(infos[1]) == 0) and (len(infos[2]) == 0):
                        source_dict[infos[0].strip()].append(sample_dict[company])
                    elif (len(infos[1]) > 0) and (len(infos[2]) == 0):
                        source_dict[infos[1].strip()].append(sample_dict[company])
                    elif (len(infos[1]) > 0) and (len(infos[2]) > 0):
                        source_dict[infos[2].strip()].append(sample_dict[company])
    
    # vote, select the one with maximum appearence
    correspond_dict = {}
    for info in source_dict:
        args, counts = np.unique(source_dict[info], return_counts=True)
        correspond_dict[info] = args[np.argmax(counts)]
    
    partial_company = set()
    for company in industry_dict:
        for year in industry_dict[company]:
            infos = re.split('-', industry_dict[company][year])
            for info in infos:
                if info in correspond_dict:
                    partial_company.add(company)
    
    # Non-replacements in original data
    print("\nManually replace the company without correpsonding reclassification")
    rest_company = set()
    for x in industry_dict.keys():
        if x not in partial_company:
            rest_company.add(x)
    print(str(len(rest_company)) + " companies are not given proper classes") 
    print("These KFIDs are listed: " + str(sorted(list(rest_company))))
    # correspond_dict = manual_reclassify(correspond_dict, rest_company)
    with open(os.path.join(root_dir, task_folder, 'correspond_dict.json'), 'w') as output_file:   
        json.dump(correspond_dict, output_file)

else:
    print("Reading dictionaries for reclassification")
    with open(os.path.join(root_dir, task_folder, 'name_dict.json'), 'r') as input_file:  
        name_dict = json.load(input_file)
    with open(os.path.join(root_dir, task_folder, 'criterion_dict.json'), 'r') as input_file:   
        criterion_dict = json.load(input_file)
    with open(os.path.join(root_dir, task_folder, 'correspond_dict.json'), 'r') as input_file:  
        correspond_dict = json.load(input_file)

print("Dictionary of naming, key: serial ID in GICS, value: corresponding name")
print(name_dict)

Reading dictionaries for reclassification


In [0]:
'''
Rewrite into csv file
'''

if not data_rewritten:
    rewritten_df = rewrite_industry(all_df, correspond_dict, criterion_dict, name_dict, industry_dict)
    rewritten_df.to_csv(os.path.join(root_dir, 'reclassified_all_data.csv'))
else:
    rewritten_df = pd.read_csv(os.path.join(root_dir, 'reclassified_all_data.csv'))



In [0]:
rewritten_df.iloc[:, :100]

,Unnamed: 0,KF_ID,CalendarYear,RegionName,OwnershipTypeDesc,NumOfEmpDesc,JobName,ReferenceLevelNum,IncumbentPointCount,Base Salary,Benefit Values,Fixed Annual Remuneration,Long Term Incentive Values,Short Term Variable Payments,Target Incentive Payment (%),Total Annual Remuneration,Total Cash,Total Direct Compensation,Total Remuneration Plus,Total Earnings,SectorNumber,SectorName,IndustryGroupNumber,IndustryGroupName,IndustryNumber,IndustryName
0,0,4,2008,42.0,0.0,8.0,Store Manager I - 1385,13,261,40000.0,12443.0,52443.0,NaN,0.0,5.0,52443.0,40000.0,40000.0,55923.0,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
1,1,4,2008,42.0,0.0,8.0,Store Manager I - 1385,13,261,41000.0,12443.0,53443.0,NaN,0.0,5.0,53443.0,41000.0,41000.0,56923.0,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
2,2,4,2008,42.0,0.0,8.0,Store Manager I - 1385,13,261,38000.0,12443.0,50443.0,NaN,0.0,5.0,50443.0,38000.0,38000.0,53923.0,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
3,3,4,2008,42.0,0.0,8.0,Store Manager I - 1385,13,261,40000.0,12443.0,52443.0,NaN,0.0,5.0,52443.0,40000.0,40000.0,55923.0,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
4,4,4,2008,42.0,0.0,8.0,Store Manager I - 1385,13,261,35000.0,12443.0,47443.0,NaN,0.0,5.0,47443.0,35000.0,35000.0,50923.0,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28397801,999995,603,2015,NaN,NaN,NaN,Sales Assistant IV,11,176,16744.0,NaN,NaN,NaN,NaN,NaN,NaN,16744.0,NaN,NaN,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
28397802,999996,603,2015,NaN,NaN,NaN,Sales Assistant IV,11,176,16744.0,NaN,NaN,NaN,NaN,NaN,NaN,16744.0,NaN,NaN,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
28397803,999997,603,2015,NaN,NaN,NaN,Sales Assistant IV,11,176,16744.0,NaN,NaN,NaN,NaN,NaN,NaN,16744.0,NaN,NaN,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail
28397804,999998,603,2015,NaN,NaN,NaN,Sales Assistant IV,11,176,16744.0,NaN,NaN,NaN,NaN,NaN,NaN,16744.0,NaN,NaN,NaN,25,consumer discretionary,2550,retailing,255040,specialty retail


In [8]:
sector_df = rewritten_df[['KF_ID','SectorNumber','SectorName','IndustryGroupNumber','IndustryGroupName','IndustryNumber','IndustryName']]
company_set = set()
industry_reclassified_dict = {}
for row_idx in range(sector_df.shape[0]):
    company = sector_df.iloc[row_idx, 0]
    if company not in company_set:
        company_set.add(company)
        industry_reclassified_dict[str(company)] = sector_df.iloc[row_idx, 1:].to_json()

print("Reclassified industry dict")
print(industry_reclassified_dict)
with open(os.path.join(root_dir, task_folder, 'industry_reclassified_dict.json'), 'w') as output_file:  
    json.dump(industry_reclassified_dict, output_file)


Reclassified industry dict
{'4': '{"SectorNumber":25.0,"SectorName":"consumer discretionary","IndustryGroupNumber":2550.0,"IndustryGroupName":"retailing","IndustryNumber":255040.0,"IndustryName":"specialty retail"}', '12': '{"SectorNumber":25.0,"SectorName":"consumer discretionary","IndustryGroupNumber":2550.0,"IndustryGroupName":"retailing","IndustryNumber":255040.0,"IndustryName":"specialty retail"}', '14': '{"SectorNumber":30.0,"SectorName":"consumer staple","IndustryGroupNumber":3020.0,"IndustryGroupName":"food beverage and tobacco","IndustryNumber":302020.0,"IndustryName":"food product"}', '17': '{"SectorNumber":25.0,"SectorName":"consumer discretionary","IndustryGroupNumber":2550.0,"IndustryGroupName":"retailing","IndustryNumber":255040.0,"IndustryName":"specialty retail"}', '21': '{"SectorNumber":40.0,"SectorName":"financials","IndustryGroupNumber":4030.0,"IndustryGroupName":"insurance","IndustryNumber":403010.0,"IndustryName":"insurance"}', '22': '{"SectorNumber":35.0,"SectorNa

In [0]:
if not paying_analysis:
    if data_rewritten:
        print("Reading reclassified data")
        all_df = pd.read_csv(os.path.join(root_dir, "reclassified_all_data.csv"))
        print("Analyze the salary based on reference levels and skill incumbent points of the jobs")
        salary_df = all_df[['KF_ID', 'CalendarYear', 'Base Salary', 'ReferenceLevelNum', 'IncumbentPointCount', 'IndustryName']]
        all_industries = pd.unique(all_df['IndustryName']).tolist()
        all_companies = pd.unique(all_df['KF_ID']).tolist()
        company_salary_dict = {}
        year_company_salary_dict = {}

        for industry in all_industries:
            if not isinstance(industry, str) :
                continue
            industry_salary_df = salary_df[salary_df['IndustryName'] == str(industry)]
            company_salary_dict[industry] = {}
            year_company_salary_dict[industry] = {}
            print("\nAnalyzing industry of " + industry)
            for company in all_companies:
                company_salary_df = industry_salary_df[industry_salary_df['KF_ID'] == company]
                all_years = pd.unique(company_salary_df['CalendarYear']).tolist()
                all_levels = pd.unique(company_salary_df['ReferenceLevelNum']).tolist()
                all_points = pd.unique(company_salary_df['IncumbentPointCount']).tolist()
                company_salary_dict[industry][company] = {}
                company_salary_dict[industry][company]['level'] = {}
                for level in sorted(all_levels):
                    level_data = company_salary_df[company_salary_df['ReferenceLevelNum'] == level]['Base Salary'].tolist()
                    mean = np.mean(level_data)
                    std = np.std(level_data)
                    company_salary_dict[industry][company]['level'][level] = (mean, std)

                company_salary_dict[industry][company]['point'] = {}
                for point in sorted(all_points):
                    point_data = company_salary_df[company_salary_df['IncumbentPointCount'] == point]['Base Salary'].tolist()
                    mean = np.mean(point_data)
                    std = np.std(point_data)
                    company_salary_dict[industry][company]['point'][point] = (mean, std)
                
                year_company_salary_dict[industry][company] = {}
                year_company_salary_dict[industry][company] = {}
                for year in all_years:
                    year_company_salary_df = company_salary_df[company_salary_df['CalendarYear'] == year]
                    all_levels = pd.unique(year_company_salary_df['ReferenceLevelNum']).tolist()
                    all_points = pd.unique(year_company_salary_df['IncumbentPointCount']).tolist()
                    year_company_salary_dict[industry][company][year] = {}
                    year_company_salary_dict[industry][company][year]['level'] = {}
                    for level in sorted(all_levels):
                        level_data = year_company_salary_df[year_company_salary_df['ReferenceLevelNum'] == level]['Base Salary'].tolist()
                        mean = np.mean(level_data)
                        std = np.std(level_data)
                        year_company_salary_dict[industry][company][year]['level'][level] = (mean, std)

                    year_company_salary_dict[industry][company][year]['point'] = {}
                    for point in sorted(all_points):
                        point_data = year_company_salary_df[year_company_salary_df['IncumbentPointCount'] == point]['Base Salary'].tolist()
                        mean = np.mean(point_data)
                        std = np.std(point_data)
                        year_company_salary_dict[industry][company][year]['point'][point] = (mean, std)
                    
        print("Saving analysis result into json")
        with open(os.path.join(root_dir, task_folder, 'company_salary_dict.json'), 'w') as output_file:  
            json.dump(company_salary_dict, output_file)
        with open(os.path.join(root_dir, task_folder, 'year_company_salary_dict.json'), 'w') as output_file:   
            json.dump(year_company_salary_dict, output_file)
    else:
        print("Please make sure you have finish the data rewritten part first!")

else:
    with open(os.path.join(root_dir, task_folder, 'company_salary_dict.json'), 'r') as input_file:  
        company_salary_dict = json.dump(input_file)
    with open(os.path.join(root_dir, task_folder, 'year_company_salary_dict.json'), 'r') as input_file:   
        year_company_salary_dict = json.dump(input_file)

Reading reclassified data
Analyze the salary based on reference levels and skill incumbent points of the jobs

Analyzing industry of specialty retail

Analyzing industry of food product

Analyzing industry of insurance

Analyzing industry of healthcare provider and service

Analyzing industry of machinery

Analyzing industry of metal and mining

Analyzing industry of chemical

Analyzing industry of pharmaceutical

Analyzing industry of hotel restaurant and leisure

Analyzing industry of container and packaging

Analyzing industry of capital market

Analyzing industry of personal product

Analyzing industry of electric utility

Analyzing industry of bank

Analyzing industry of healthcare equipment and supply

Analyzing industry of road and rail

Analyzing industry of diversified consumer service

Analyzing industry of construction and engineering

Analyzing industry of beverage

Analyzing industry of energy equipment and service

Analyzing industry of aerospace and defense

Analyzing in